## Recency, Frequency, Monetary Value analysis


In [2]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv('customers.csv')

### Calculate spend quartiles (q=4)

In [7]:

spend_quartile = pd.qcut(data['Spend'], q=4, labels=range(1,5))


data['Spend_Quartile'] = spend_quartile


print(data.sort_values('Spend'))

   Unnamed: 0  CustomerID  Spend Spend_Quartile
0           0           0    137              1
2           2           2    172              1
7           7           7    229              2
5           5           5    233              2
6           6           6    244              3
4           4           4    303              3
1           1           1    335              4
3           3           3    355              4


### Calculate recency deciles (q=4)

In [ ]:

r_labels = list(range(4, 0, -1))


recency_quartiles = pd.qcut(data['Recency_Days'], q=4, labels=r_labels)


data['Recency_Quartile'] = recency_quartiles 


print(data.sort_values('Recency_Days'))

### Calculate RFM values

In [4]:
online = pd.read_csv('online.csv')

In [5]:
snapshot_date = pd.Timestamp(2011,12,10)

In [6]:
snapshot_date

Timestamp('2011-12-10 00:00:00')

In [7]:
online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSum
0,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25,2.10,14286,United Kingdom,12.60
1,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20,1.45,16360,United Kingdom,1.45
2,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14,3.75,13933,United Kingdom,22.50
3,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23,2.10,17290,United Kingdom,2.10
4,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25,5.95,17663,United Kingdom,17.85


In [8]:
online['InvoiceDate'] = pd.to_datetime(online.InvoiceDate)

In [9]:

datamart = online.groupby(['CustomerID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'TotalSum': 'sum'})


datamart.rename(columns={'InvoiceDate': 'Recency',
                         'InvoiceNo': 'Frequency',
                         'TotalSum': 'MonetaryValue'}, inplace=True)


print(datamart.head())

            Recency  Frequency  MonetaryValue
CustomerID                                   
12747             3         25         948.70
12748             1        888        7046.16
12749             4         37         813.45
12820             4         17         268.02
12822            71          9         146.15


### Calculate 3 groups for recency and frequency

In [10]:

r_labels = range(3, 0, -1); f_labels = range(1, 4)


r_groups = pd.qcut(datamart['Recency'], q=3, labels=r_labels)


f_groups = pd.qcut(datamart['Frequency'], q=3, labels=f_labels)


datamart = datamart.assign(R=r_groups.values, F=f_groups.values)

In [12]:
r_labels

range(3, 0, -1)

In [13]:
f_labels

range(1, 4)

In [11]:
datamart.head()

,Recency,Frequency,MonetaryValue,R,F
CustomerID,,,,,
12747,3,25,948.70,3,3
12748,1,888,7046.16,3,3
12749,4,37,813.45,3,3
12820,4,17,268.02,3,3
12822,71,9,146.15,2,2
